In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
from typing import Tuple

In [2]:
def cond(m: np.array):
    M_inv = np.linalg.inv(m)
    return norm(M_inv) * norm(m)

def sin(z: np.ndarray, i : int, j : int):
    return - z[j] / np.sqrt(z[i]**2 + z[j]**2)

def cos(z: np.ndarray, i : int, j : int):
    return z[i] / np.sqrt(z[i]**2 + z[j]**2)

def get_t(i: int, j: int, z: np.ndarray):
    n = z.shape[0]
    cos_phi = cos(z, i, j)
    sin_phi = sin(z, i, j)
    T = np.identity(n)
    T[i, i] = cos_phi
    T[j, j] = cos_phi
    T[j, i] = sin_phi
    T[i, j] = -sin_phi
    return T


def get_q(A: np.ndarray) -> np.ndarray:
    n = A.shape[0]
    A_copy = A.copy()
    M = np.eye(n)
    for i in range(n):
        for j in range(i + 1, n):
            T = get_t(i, j, A_copy[:, i])
            T_inv = np.linalg.inv(T)
            M = np.matmul(M, T_inv)        #T.T)
            A_copy = np.matmul(T, A_copy)
    return M

def get_qr(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Q = get_q(A)
    return Q, np.matmul(Q.T, A)

In [3]:
def solve_trig(A: np.ndarray, b: np.ndarray):
    n = A.shape[0]
    A = np.c_[A, b]
    solutions = np.array([A[n - 1][n] / A[n - 1][n - 1]])
    for i in range(n - 2, -1, -1):
        x = A[i][n] - np.dot(solutions, A[i, (i+1):n])
        solutions = np.insert(solutions, 0, x / A[i, i])
    solutions = solutions.T
    return solutions

def solve_with_qr(A: np.ndarray, u: np.ndarray):
    n = A.shape[0]
    Q, R = get_qr(A)
    new_u = np.matmul(Q.T, u)
    x = solve_trig(R, new_u)
    stat = pd.DataFrame({
        'Crit Q': [cond(Q)],
        'Crit R': [cond(R)],
        'Crit A': [cond(A)]
    })

    return x, stat

Test

In [4]:
A = np.array([
    [1, 0.99],
    [0.99, 0.98]
])

b = np.array([
    [1.99],
    [1.97]
])

x, stat = solve_with_qr(A, b)
stat

,Crit Q,Crit R,Crit A
0,2.0,39206.0,39206.0


In [5]:
x

array([1., 1.])

In [6]:
from scipy.linalg import hilbert

h1 = hilbert(10)
u = np.matmul(h1, np.ones(10))
_, stat = solve_with_qr(h1, u)
stat

,Crit Q,Crit R,Crit A
0,10.0,1.633240e+13,1.633232e+13


In [7]:
h2 = hilbert(50)
u = np.matmul(h2, np.ones(50))
_, stat = solve_with_qr(h2, u)
stat

,Crit Q,Crit R,Crit A
0,50.0,9.828733e+18,1.502597e+19
